<a href="https://colab.research.google.com/github/saja13-zh/NLP-project/blob/main/nlp_project_finaled_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **load the dataset and preview it**

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("hard")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset['train'].data

MemoryMappedTable
text: string
label: int64
----
text: [["“ممتاز”. النظافة والطاقم متعاون. ","استثنائي. سهولة إنهاء المعاملة في الاستقبال. لاشيئ","استثنائي. انصح بأختيار الاسويت و بالاخص غرفه رقم 801. نوعية الارضيه","“استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق 2 نجمه ","جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كان حوض السباحه لايعمل في هذي الفتره حسب كلامهم يقولوا فيه صيانه والله اعلم",...,"“غير جيد وغالي”. . الغرف ضيقة والمصاعد قديمه واثاث المنتجع قديم","“كنت أتمنى الأفضل”. لاشيء. عدم المام موظفي الاستقبال بالمعلومات الكافية ولغتهم العربية ركيكة","“للراحة والاسترخاء والاستمتاع.. سراي عجمان هو المفتاح...”. شكرًا على لطافتكم... بالفعل استمتعت بكل دقيقة .. بالتأكيد لن تكون الزيارة الاولى ... وسوف اعتمدها كمكان  لراحتي ووقت للاسترخاء ... شكرًا جدا على التعامل الراقي من قبل جميع الموظفين... وشكراً على الخدمات المنفردة عن غيرها...لي عودة فالقريب العاجل... تحياتي... ","“كانت اجازه جدا جميله”. ماشالله كل شي فيه حلو وخصوصي انو عائلي ممتاز. ","استثنائي. كل شي ممتاز من خدمة ونظافة وطعام. "]

**split the dataset into text and labels**

In [ ]:
text_data = dataset['train'].data['text']
labels = dataset['train'].data['label']

for i in range(min(5, len(text_data))):
    print(f"Text data {i+1}: {text_data[i]}")
    print(f"Labels {i+1}: {labels[i]}")
    print("----------------------")


Text data 1: “ممتاز”. النظافة والطاقم متعاون. 
Labels 1: 1
----------------------
Text data 2: استثنائي. سهولة إنهاء المعاملة في الاستقبال. لاشيئ
Labels 2: 4
----------------------
Text data 3: استثنائي. انصح بأختيار الاسويت و بالاخص غرفه رقم 801. نوعية الارضيه
Labels 3: 4
----------------------
Text data 4: “استغرب تقييم الفندق كخمس نجوم”. لا شي. يستحق 2 نجمه 
Labels 4: 0
----------------------
Text data 5: جيد. المكان جميل وهاديء. كل شي جيد ونظيف بس كان حوض السباحه لايعمل في هذي الفتره حسب كلامهم يقولوا فيه صيانه والله اعلم
Labels 5: 3
----------------------


**data cleaning use tnkeeh**

In [ ]:
!pip install tnkeeh

In [ ]:
import tnkeeh as tn

cleaner = tn.Tnkeeh(remove_diacritics = True,remove_special_chars=True,remove_english=True,
                 normalize=True,remove_tatweel=True,remove_repeated_chars=True,remove_html_elements=True,remove_links=True,remove_twitter_meta=True,remove_long_words=True   )

cleaned_dataset = cleaner.clean_hf_dataset(dataset, 'text')

In [ ]:
for i in range(min(5, len(cleaned_dataset['train']['text']))):
    print(f"cleaned dataset {i+1}: {cleaned_dataset['train']['text'][i]}")


cleaned dataset 1:  ممتاز النظافة والطاقم متعاون 
cleaned dataset 2: استثنائي سهولة إنهاء المعاملة في الاستقبال لاشيئ
cleaned dataset 3: استثنائي انصح بأختيار الاسويت و بالاخص غرفه رقم 801 نوعية الارضيه
cleaned dataset 4:  استغرب تقييم الفندق كخمس نجوم لا شي يستحق 2 نجمه 
cleaned dataset 5: جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان حوض السباحه لايعمل في هذي الفتره حسب كلامهم يقولوا فيه صيانه والله اعلم


Replace all forms of letter alef-with-hamza (أ، إ، آ) with the normalized form (ا).
and (ة) with (ه)

In [ ]:
def normalizeArabic(text):
    if isinstance(text, str):
        normalized_text = text.replace("إ", "ا").replace("أ", "ا").replace("آ", "ا")
        normalized_text = normalized_text.replace("ة", "ه").replace("ى", "ي")
        normalized_text = normalized_text.replace("ؤ", "ء")
        return normalized_text
    else:
        return text


normalized_texts = [normalizeArabic(str(text)) for text in cleaned_dataset['train']['text']]

for i in range(min(5, len(normalized_texts))):
    print(f"normalized dataset {i+1}: {normalized_texts[i]}")



normalized dataset 1:  ممتاز النظافه والطاقم متعاون 
normalized dataset 2: استثنائي سهوله انهاء المعامله في الاستقبال لاشيئ
normalized dataset 3: استثنائي انصح باختيار الاسويت و بالاخص غرفه رقم 801 نوعيه الارضيه
normalized dataset 4:  استغرب تقييم الفندق كخمس نجوم لا شي يستحق 2 نجمه 
normalized dataset 5: جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان حوض السباحه لايعمل في هذي الفتره حسب كلامهم يقولوا فيه صيانه والله اعلم


In [ ]:
import nltk
from nltk.corpus import stopwords

# Download NLTK's stopwords for Arabic
nltk.download('stopwords')

# Load the stopwords for Arabic
stopwords_arabic = set(stopwords.words('arabic'))
#remove what needed from stop words
want_word={'لا','نعم','ليس','إلا','كلا','ولا'}
stopwords=[word for word in stopwords_arabic if word not in want_word]
print(stopwords)
print(len(stopwords))
print(len(stopwords_arabic))

['يناير', 'لكيلا', 'طفق', 'ثمانمئة', 'خمسمئة', 'الآن', 'عوض', 'كلما', 'اللائي', 'ذو', 'ف', 'فلس', 'غالبا', 'تارة', 'ذهب', 'تلكما', 'حيَّ', 'بَلْهَ', 'أنبأ', 'لات', 'كلّما', 'كأي', 'لولا', 'أيا', 'كأن', 'حسب', 'هاتان', 'بكما', 'أحد', 'شمال', 'مثل', 'ثمانين', 'لبيك', 'ساء', 'هذه', 'هن', 'ثاني', 'ذيت', 'هنا', 'ريال', 'غين', 'تفعلون', 'حتى', 'إياه', 'إزاء', 'وجد', 'خامس', 'عشرة', 'حبذا', 'جويلية', 'تسع', 'كاف', 'علق', 'هَاتِه', 'بئس', 'لسنا', 'هلم', 'كلتا', 'لهم', 'به', 'حقا', 'ة', 'أخذ', 'أيضا', 'هنالك', 'بك', 'خمسين', 'جلل', 'ارتدّ', 'بهما', 'ع', 'دونك', 'هَذَيْنِ', 'أقل', 'عَدَسْ', 'كى', 'ابتدأ', 'يورو', 'ألفى', 'أخٌ', 'اللواتي', 'ئ', 'بضع', 'شباط', 'فيه', 'خاصة', 'أنتما', 'ين', 'باء', 'الذي', 'يفعلان', 'أجمع', 'خمس', 'ض', 'مائة', 'إذما', 'تلك', 'خميس', 'لك', 'ومن', 'طَق', 'سبعة', 'خمسة', 'حادي', 'غدا', 'كسا', 'إي', 'سادس', 'صبر', 'أينما', 'ذي', 'بعد', 'سرعان', 'تحت', 'تين', 'خلف', 'كلَّا', 'نون', 'خمسمائة', 'جعل', 'ليست', 'أنتِ', 'عند', 'معاذ', 'مكانكنّ', 'ما أفعله', 'تاسع', 'كما', 'عم

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **stop word removal**

In [ ]:
# Function to remove stopwords
def remove_stopwords(text):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word.lower() not in stopwords]
    return ' '.join(filtered_tokens)

# Apply stopword removal to your normalized texts
normalized_texts_without_stopwords = [remove_stopwords(text) for text in normalized_texts]

# Print original text and text after stopword removal
for i in range(min(5, len(normalized_texts_without_stopwords))):
    print(f"Original Text {i+1}: {normalized_texts[i]}")
    print(f"Text after Stopword Removal {i+1}: {normalized_texts_without_stopwords[i]}")
    print("----------------------")


Original Text 1:  ممتاز النظافه والطاقم متعاون 
Text after Stopword Removal 1: ممتاز النظافه والطاقم متعاون
----------------------
Original Text 2: استثنائي سهوله انهاء المعامله في الاستقبال لاشيئ
Text after Stopword Removal 2: استثنائي سهوله انهاء المعامله الاستقبال لاشيئ
----------------------
Original Text 3: استثنائي انصح باختيار الاسويت و بالاخص غرفه رقم 801 نوعيه الارضيه
Text after Stopword Removal 3: استثنائي انصح باختيار الاسويت بالاخص غرفه رقم 801 نوعيه الارضيه
----------------------
Original Text 4:  استغرب تقييم الفندق كخمس نجوم لا شي يستحق 2 نجمه 
Text after Stopword Removal 4: استغرب تقييم الفندق كخمس نجوم لا شي يستحق 2 نجمه
----------------------
Original Text 5: جيد المكان جميل وهاديء كل شي جيد ونظيف بس كان حوض السباحه لايعمل في هذي الفتره حسب كلامهم يقولوا فيه صيانه والله اعلم
Text after Stopword Removal 5: جيد المكان جميل وهاديء شي جيد ونظيف حوض السباحه لايعمل الفتره كلامهم يقولوا صيانه والله اعلم
----------------------


# **Tokenize text**

In [ ]:
import nltk
nltk.download('punkt')

# Function to tokenize text
def tokenize_text(text):
    tokens =text.split()
    return tokens

# Tokenize each text after stopword removal
tokenized_texts = [tokenize_text(text) for text in normalized_texts_without_stopwords]

# Print the tokenized texts
for tokens in range(min(5, len(tokenized_texts))):
    print(tokenized_texts[tokens])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['ممتاز', 'النظافه', 'والطاقم', 'متعاون']
['استثنائي', 'سهوله', 'انهاء', 'المعامله', 'الاستقبال', 'لاشيئ']
['استثنائي', 'انصح', 'باختيار', 'الاسويت', 'بالاخص', 'غرفه', 'رقم', '801', 'نوعيه', 'الارضيه']
['استغرب', 'تقييم', 'الفندق', 'كخمس', 'نجوم', 'لا', 'شي', 'يستحق', '2', 'نجمه']
['جيد', 'المكان', 'جميل', 'وهاديء', 'شي', 'جيد', 'ونظيف', 'حوض', 'السباحه', 'لايعمل', 'الفتره', 'كلامهم', 'يقولوا', 'صيانه', 'والله', 'اعلم']


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

clean_texts = [' '.join(text) for text in tokenized_texts]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(clean_texts)
# X is a sparse matrix of shape (n_samples, n_features)
print(X.shape)
print(X[0])

(105698, 118419)
  (0, 80972)	0.5590099036743219
  (0, 97045)	0.6887139971409053
  (0, 27718)	0.33317470269260896
  (0, 87022)	0.319649143940531


In [ ]:
for i in range(min(5, len(clean_texts))):
    print(f"Clean Text {i+1}: {X[i]}")
    print("----------------------")


Clean Text 1:   (0, 80972)	0.5590099036743219
  (0, 97045)	0.6887139971409053
  (0, 27718)	0.33317470269260896
  (0, 87022)	0.319649143940531
----------------------
Clean Text 2:   (0, 70196)	0.49825177596702147
  (0, 8482)	0.19536029174634315
  (0, 25299)	0.4154768420940183
  (0, 29994)	0.5082172479277967
  (0, 55552)	0.4693431009849115
  (0, 4498)	0.24978150512925987
----------------------
Clean Text 3:   (0, 8191)	0.27183059233565066
  (0, 91525)	0.29629588709424837
  (0, 1094)	0.47814087848128434
  (0, 52809)	0.2564752834321436
  (0, 62486)	0.1729541689006465
  (0, 31833)	0.32764807807020724
  (0, 8849)	0.47814087848128434
  (0, 31162)	0.3617635138401959
  (0, 29792)	0.1592262533730339
  (0, 4498)	0.14767416391414442
----------------------
Clean Text 4:   (0, 89748)	0.33096787096105884
  (0, 115555)	0.264307758616284
  (0, 57758)	0.16550426594427647
  (0, 69191)	0.13802887869790276
  (0, 89770)	0.24208100191948312
  (0, 67953)	0.5847856949249174
  (0, 20937)	0.12959960071604348
  (

In [ ]:
import numpy as np

labels = np.array(labels)
print(labels.shape)
print(X.shape)

(105698,)
(105698, 118419)


In [ ]:
for i in range(min(5, len(clean_texts))):
    print(f"Clean Text {i+1}: {labels[i]}")

def sentiment(n):
    return 1 if n >= 3 else 0

last_labels = [sentiment(label) for label in labels]
last_labels = np.array(last_labels)
print("----------------------")
for i in range(min(5, len(clean_texts))):
    print(f"Clean Text {i+1}: {last_labels[i]}")


Clean Text 1: 1
Clean Text 2: 4
Clean Text 3: 4
Clean Text 4: 0
Clean Text 5: 3
----------------------
Clean Text 1: 0
Clean Text 2: 1
Clean Text 3: 1
Clean Text 4: 0
Clean Text 5: 1


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, last_labels, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
print(f"X_train data type: {type(X_train)}")
print(f"y_train data type: {type(y_train)}")

(84558, 118419)
(21140, 118419)
(84558,)
(21140,)
X_train data type: <class 'scipy.sparse._csr.csr_matrix'>
y_train data type: <class 'numpy.ndarray'>


In [ ]:
import tensorflow as tf

X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()


In [ ]:

print(f"X_train data type: {type(X_train_dense)}")
print(f"y_train data type: {type(y_train)}")
print(f"X_test data type: {type(X_test_dense)}")
print(f"y_test data type: {type(y_test)}")

X_train data type: <class 'numpy.ndarray'>
y_train data type: <class 'numpy.ndarray'>
X_test data type: <class 'numpy.ndarray'>
y_test data type: <class 'numpy.ndarray'>


لسا نحتاج نعدل عليه اذا ضبطت الداتا

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense


# Define the model
model = Sequential()
model.add(Embedding(input_dim=X_train_dense.shape[1], output_dim=100, input_length=100))  # Adjust input_length to your desired sequence length
model.add(LSTM(units=128))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

model.build(input_shape=(None, X_train_dense.shape[1]))
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          11841900  
                                                                 
 lstm (LSTM)                 (None, 128)               117248    
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 2)                 130       
                                                                 
Total params: 11967534 (45.65 MB)
Trainable params: 11967534 (45.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Train the model
model.fit(X_train_dense, y_train, epochs=5, batch_size=32)


In [ ]:
# Evaluate model on test data
score = model.evaluate(X_test, y_test, verbose=1)

# Calculate precision, recall, F1 score
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
y_test_argmax = np.argmax(y_test, axis=1)

print('Precision:', precision_score(y_test_argmax, y_pred, average='weighted'))
print('Recall:', recall_score(y_test_argmax, y_pred, average='weighted'))
print('F1 score:', f1_score(y_test_argmax, y_pred, average='weighted'))